<a href="https://colab.research.google.com/github/LEEINSEO-0118/Seed-size-calculator/blob/main/CalculateSize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import


# Calculate Functions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import seaborn as sns
from tqdm import tqdm
import re

## Standard Template
사이즈 비교의 대상이 되는 템플릿


In [3]:
# 실제로 사용한 30mm x 30mm 사이즈의 종이 템플릿을 30배 스케일로 늘려 템플릿 생성
SCALE = 30
PAPER_W = 30 * SCALE
PAPER_H = 30 * SCALE

paper_coords = np.float32([[0,0],                # Top left
                           [PAPER_W,0],          # Top right
                           [0,PAPER_H],          # Bottom left
                           [PAPER_W,PAPER_H]])   # Bottom right

## Load Image and Show Image

In [4]:
def load_image(path, scale = 0.7):
    img = cv2.imread(path)
    img_resized = cv2.resize(img, (0,0), None, scale, scale)
    return img_resized

def show_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.xticks([])
    plt.yticks([]) # 얘네 넣으면 축에 index가 사라지네?
    plt.imshow(img)
    plt.show()

## Image Preprocess
이미지 -> 그레이 스케일 -> Blur 처리 -> edge 추출 -> edge굵게 및 선명하게

In [5]:
def preprocess_image(img, thresh_1=57, thresh_2=232):
    img_gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)      # 그레이 스케일 변환
    img_blur  = cv2.GaussianBlur(img_gray, (5,5), 1)       # 필터처리
    img_canny = cv2.Canny(img_blur, thresh_1, thresh_2)    # edge detector

    kernel = np.ones((3,3))    #4
    img_dilated = cv2.dilate(img_canny, kernel, iterations=1)    # edge 라인을 굵게
    img_closed = cv2.morphologyEx(img_dilated, cv2.MORPH_CLOSE,
                                  kernel, iterations=4)          # edge 내부 small holes 제거, edge를 명확하게

    img_preprocessed = img_closed.copy()

    img_each_step = {'img_dilated': img_dilated,
                     'img_canny'  : img_canny,
                     'img_blur'   : img_blur,
                     'img_gray'   : img_gray}

    return img_preprocessed, img_each_step

## Find Contours
Contour를 통해 모서리 추출

In [6]:
def find_contours(img_preprocessed, img_original, epsilon_param=0.04):
    contours, hierarchy = cv2.findContours(image=img_preprocessed,
                                           mode=cv2.RETR_EXTERNAL, # for outermost contours
                                           method=cv2.CHAIN_APPROX_NONE)  # find contour

    img_contour = img_original.copy() # copy origin image
    cv2.drawContours(img_contour, contours, -1, (203,192,255), 6)  # draw contour on copied origin image

    polygons = []
    for contour in contours:
        epsilon = epsilon_param * cv2.arcLength(curve=contour,
                                                closed=True)  # epsilon 값이 작으면 많은 코너를 탐지한다.
                                                              # 사이즈 기준 객체의 정확한 코너를 찾기 위해서 epsilon 값을 설정하는 것에 주의해야 함
        polygon = cv2.approxPolyDP(curve=contour,
                                   epsilon=epsilon, closed=True)  # contour를 통해 코너(Curves) 추정
        # print(polygon.shape)
        polygon = polygon.reshape(4, 2)  # original output of cv2.approxPolyDP() is in the format (number of corners, 1, 2), where the middle axis is irrelevant in our case. Hence, we can safely discard it.
        # print(polygon.shape)
        # print(polygon)
        polygons.append(polygon)
        for point in polygon:
            img_contour = cv2.circle(img=img_contour, center=point,
                                     radius=8, color=(0,240,0),
                                     thickness=-1)
        break

    return polygons, img_contour

모서리 좌표 순서 정렬

In [7]:
def reorder_coords(polygon):
    rect_coords = np.zeros((4, 2)) # 각 모서리 구해서 넣기

    # 왼쪽 상단이 0, 0
    # 좌표 합 가장 작은 것 -> 좌측 상단
    # 좌표 합 가장 큰 것 -> 우측 하단
    add = polygon.sum(axis=1) # 컬럼을 기준으로 합치기
    rect_coords[0] = polygon[np.argmin(add)]    # Top left # 최소값의 index 반환
    rect_coords[3] = polygon[np.argmax(add)]    # Bottom right

    # 차 y - x
    # 좌표 차 가장 작은 것 -> 우측 상한
    # 좌표 차 가장 큰 것 -> 좌측 하단
    subtract = np.diff(polygon, axis=1)
    rect_coords[1] = polygon[np.argmin(subtract)]    # Top right
    rect_coords[2] = polygon[np.argmax(subtract)]    # Bottom left

    return rect_coords

## Perspective Transform

호모그래피 추출 및 원근 변환

In [8]:
def warp_image(rect_coords, paper_coords, img_original, pad=25):

    # 실제 이미지에서 template으로 변환하는 homography 생성
    matrix, _ = cv2.findHomography(rect_coords, paper_coords)

    # trasform
    img_warped = cv2.warpPerspective(img_original, matrix,
                                        (PAPER_W, PAPER_H))

    # 이미지가 원하지 않는 영역을 가지기에 pad 만큼 그것을 제거
    # 음.. 꼭 해야하나..?
    # -> 이거 안하니까 계산기 corner가 안 찾아짐... 미친.. 왜?
    warped_h = img_warped.shape[0]
    warped_w = img_warped.shape[1]
    img_warped = img_warped[pad:warped_h-pad, pad:warped_w-pad]

    return img_warped

## Find Ellipse contours

종실 형태를 기준으로 타원을 형성 -> 타원을 기준으로 Rotated된 직사각형 생성 -> 종실의 크기 측정

In [9]:
def find_ellipse_contours(img_preprocessed, img_original):

    thresh_1, thresh_2 = 57, 232
    canny_output = cv2.Canny(img_preprocessed, thresh_1, thresh_2)
    contours, hierarchy = cv2.findContours(image=img_preprocessed,
                                           mode=cv2.RETR_EXTERNAL, # for outermost contours
                                           method=cv2.CHAIN_APPROX_NONE)  # find contour

    # Find the rotated rectangles and ellipses for each contour
    minRect = [None]*len(contours)
    for i, c in enumerate(contours):
        minRect[i] = cv2.minAreaRect(c)

    img_contour = img_original.copy()

    polygons = cv2.boxPoints(minRect[i])
    polygons = np.intp(polygons) #np.intp: Integer used for indexing (same as C ssize_t; normally either int32 or int64)
    cv2.drawContours(img_contour, [polygons], -1, (255,0,0), 3)

    for point in polygons:
        img_contour = cv2.circle(img=img_contour, center=point, radius=8, color=(0,240,0), thickness=-1)

    return img_contour, polygons

## Calculate size

In [10]:
def calculate_sizes(polygons):
    sizes = []
    lens = len(polygons)
    for i in range(lens):
        for j in range(i+1, lens):
            dist = cv2.norm(polygons[i], polygons[j], cv2.NORM_L2)
            sizes.append([dist, [i, j]]) # 길이와 두 점의 index 저장

    sorted_sizes = sorted(sizes, key=lambda x: x[0])

    result = [sorted_sizes[0], sorted_sizes[3]]
    return result

In [11]:
def convert_to_mm(sizes_pixel, img_warped):
    warped_h = img_warped.shape[0]
    warped_w = img_warped.shape[1]

    scale_h = PAPER_H / warped_h    # 높이 비율
    scale_w = PAPER_W / warped_w    # 너비 비율


    size_pixel_h, size_pixel_w = sizes_pixel[0][0], sizes_pixel[1][0]
    size_mm_h = size_pixel_h * scale_h / SCALE
    size_mm_w = size_pixel_w * scale_w / SCALE

    sizes_mm = [[size_mm_h, sizes_pixel[0][1]], [size_mm_w, sizes_pixel[1][1]]]

    return sizes_mm

## Print size

In [12]:
def write_size(polygons, sizes, img_warped):

    img_result = img_warped.copy()
    for size, idx in (sizes):
        cv2.line(img_result, polygons[idx[0]], polygons[idx[1]], (0,0,255), 4)
        cv2.putText(img_result, f'{np.float32(size):.2f}mm',
                    (polygons[idx[0]][0]-50, polygons[idx[0]][1]+100),
                    cv2.FONT_HERSHEY_DUPLEX, 0.9, (0,0,255), 1)

    return img_result

## Total Function

In [29]:
def measure_size(path, img_original_scale=0.7,
                 PAPER_W=30, PAPER_H=30, SCALE=30,
                 paper_eps_param=0.04,canny_thresh_1=57, canny_thresh_2=232):

    PAPER_W = PAPER_W * SCALE
    PAPER_H = PAPER_H * SCALE

    # Loading and preprocessing original image.
    img_original = load_image(path=path, scale=img_original_scale)
    img_preprocessed, img_each_step = preprocess_image(img_original,
                                                       thresh_1=canny_thresh_1,
                                                       thresh_2=canny_thresh_2)

    # Finding paper contours and corners.
    polygons, img_contours = find_contours(img_preprocessed,
                                           img_original,
                                           epsilon_param=paper_eps_param)

    # show_image(img_contours)

    # Reordering paper corners.
    rect_coords = np.float32(reorder_coords(polygons[0]))

    # Warping image according to paper contours.
    paper_coords = np.float32([[0,0],
                               [PAPER_W,0],
                               [0,PAPER_H],
                               [PAPER_W,PAPER_H]])
    img_warped = warp_image(rect_coords, paper_coords, img_original)

    # Preprocessing the warped image.
    img_warped_preprocessed, _ = preprocess_image(img_warped)

    # Finding seed rectangle
    img_contours_warped, polygons = find_ellipse_contours(img_warped_preprocessed, img_warped)


    # Edge langth calculation.
    seed_size = calculate_sizes(polygons)
    sizes_mm = convert_to_mm(seed_size, img_warped)
    img_result = write_size(polygons, sizes_mm, img_warped)

    return img_result, sizes_mm

# Calculate Images

## Make DataFrame

In [14]:
# get seed data
file_name = 'data.xlsx'
path = f'/content/drive/MyDrive/졸업논문/data/seed_data/{file_name}'
data = pd.read_excel(path)
data = data.drop('Unnamed: 0', axis = 1)

In [15]:
data

,KNU,Name,Weight,long,short
0,1,금화재래,0.17,7.976471,7.341176
1,3,Hongkong,0.19,7.844192,7.419240
2,6,K.L.S 901-1,0.16,8.841370,7.052203
3,7,금강대립,0.35,11.413159,8.622245
4,8,광교,0.21,8.058351,7.868689
...,...,...,...,...,...
395,611,경상대-2007-14354,0.43,0.000000,0.000000
396,612,경상대-2007-14477,0.17,7.905882,7.482353
397,613,경상대-2007-14492,0.13,0.000000,0.000000
398,614,경상대-2007-14495,0.18,8.752941,7.623529


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   KNU     400 non-null    int64  
 1   Name    400 non-null    object 
 2   Weight  400 non-null    float64
 3   long    400 non-null    float64
 4   short   400 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 15.8+ KB


In [17]:
data.describe()

,KNU,Weight,long,short
count,400.000000,400.000000,400.000000,400.000000
mean,339.250000,0.204750,4.924014,4.358804
std,179.420405,0.095484,4.365786,3.862031
min,1.000000,0.040000,0.000000,0.000000
25%,184.750000,0.140000,0.000000,0.000000
50%,372.500000,0.190000,6.811765,6.154643
75%,497.250000,0.250000,8.788395,7.722920
max,615.000000,0.580000,12.837990,11.067393


## Get Image lists

In [18]:
path = f'/content/drive/MyDrive/졸업논문/data/fail_image/'
file_lists = os.listdir(path)
file_lists.sort()
if len(data[data.long == 0].KNU) == len(file_lists):
    print('Correct')

Correct


In [184]:
# 이미지 labeling

for label, file_name in tqdm(zip(data[data.long == 0].KNU.values, file_lists)):
    path = f'/content/drive/MyDrive/졸업논문/data'
    shutil.copyfile(f'{path}/fail_image/{file_name}', f'{path}/origin_image_sort/KNU_{label}.jpeg')

168it [00:11, 14.52it/s]


In [20]:
path = f'/content/drive/MyDrive/졸업논문/data/origin_image_sort/'
file_lists = os.listdir(path)
if len(data.KNU) == len(file_lists):
    print('Correct')

Correct


## Calculate


In [22]:
fail_ary = []
correct_cnt = 0
fail_cnt = 0
for file_name in tqdm(file_lists):
    path = f'/content/drive/MyDrive/졸업논문/data'
    label = re.findall(r'\d+', file_name)
    사이즈 측정된 이미지와 사이즈 받기
    try :
        cal_image, sizes = measure_size(f'{path}/origin_image_sort/{file_name}', paper_eps_param = 0.08) # 작을 수록 많은 값을 코너를 탐지
        size_long = max(sizes[0][0], sizes[1][0])
        size_short = min(sizes[0][0], sizes[1][0])
        idx = data[data.KNU == label].index

        data.loc[idx, 'long'] = size_long
        data.loc[idx, 'short'] = size_short
        cv2.imwrite(f'{path}/calc_image/{file_name}', cal_image)

        correct_cnt += 1
    except:
        fail_cnt += 1
        fail_ary.append(file_name)
print(correct_cnt, fail_cnt)

100%|██████████| 168/168 [01:16<00:00,  2.20it/s]

58 110


## Save Data

In [25]:
path = f'/content/drive/MyDrive/졸업논문/data/seed_data/'
data.to_excel(excel_writer=path+'data.xlsx')